# Visual Inspection  of Health Center -  Cardsheet  and CG Generated  Data

This is a part of a Ph.D research project entitled with "Developing clincal decision support point of care instrument for low resource settings". It's supervised by  Bart Jansen (Prof), Frank Verbeke (Prof), Jan Cornelis (Prof), Girum Ketema (Ass.Prof) and Demesewu Amenu (Associate Prof). 
  

# Ehtiopian Primary Health System (Highlights) 

1. The public health  care system in Ethiopia is structured around the concept of a “health network model” that uses a three tiered health care delivery levels namely primary, secondary and tertiary levels with defined populations to be served at each level.  

2. For rural settings at the base is the primary health care unit that is a health centre with five satellite health posts catering to a population of 25,000; followed by  a primary hospital, serving a population of  60,000 to 100,000; and next a general  hospital, providing services to 1 up to 1.5 million beneficiaries.  For urban setting at the base is a health center serving 40,000 people, followed by a general hospital as in the rural setting. At the apex of both structures is specialized hospital which serves 3.5 to 5 million people



In [ ]:
#importing the required labraries 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import holoviews as hv
import datashader as ds
import dask as dask
import bokeh as bokeh
#import chart_studio.plotly as py
import holoviews as hv
import datashader as ds
import dask as dask
import bokeh as bokeh
import cufflinks as cf


import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
#init_notebook_mode(connected=True)
import ipywidgets as widgets
from ipywidgets import SelectionSlider
from ipywidgets import interact, interact_manual
import os
from IPython.display import Image


# Reference file used w  

In [ ]:
#to be added soon

# Facts and challenges of the healthcenter service 

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('rsOeJFYro2Y', start=13)

# Q ? 

Inspecting the card-sheet and CG to uncover and answer:
1. How are the diseases responsible for a certain complaint? 
2. Whcih disease are serious and treatble?
3. How the treatment and referral decision is made?  
4. What are the relevant findings form each or these diseases?


# Part -I : Health Center - Cardsheet 

1. Import the CSV cardsheet dataset

In [ ]:
#import the health cetner RRL (wit ICPC-2 and ICD11 format)

#pregancy detail dataset: PregnacyDetailbyCat-Table 1.csv
#All hc dataset: AllHCDataset-Table 1.csv 
#Aug 2 oct dataset:Aug2OctPart2-Table 1.csv
#april2june dataset: April2JunePart1-Table 1.csv

hcdf = pd.read_csv("AllHCDataset-Table 1.csv", encoding='utf-8')


2. Splitting TR_Date

In [ ]:
#Splitting the date into year, month and date
hcdf['TR_Date'] = pd.to_datetime(hcdf.TR_Date,errors='coerce')
hcdf['Day'],hcdf['Month'],hcdf['Year'] = hcdf.TR_Date.dt.day, hcdf.TR_Date.dt.month, hcdf.TR_Date.dt.year

#display the new RRL list  
#interacting with the raw healthcenter CSV dataset
@interact
def show_referral_by_age(column=["SN"],x=(5,1000,5)):
    return hcdf.loc[hcdf[column] >x]

In [ ]:
hcdf.shape

2. Filiter by ICPC2_Classfication

In [ ]:
#Health center cardsheet data fliter by ICPC2  and disease name

#Step-1: a generic fucntion is rquired to capture uniques unique values 
ALL = 'ALL' 
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

#Step-2: intialize a variable to capture the ICPC2 and Disease 

output = widgets.Output()

icpc2Classfication = widgets.Dropdown(options =unique_sorted_values(hcdf.ICPC2_Classfication.astype(str)), description="Ref. Reason")
diseaseName = widgets.Dropdown(options =unique_sorted_values(hcdf.Disease_Name),description="Feedback: ")

#Step-3: Write an event-handeler function     
def healthCenter_flitering(icpc2, diseasename):
    output.clear_output()

    if (icpc2 == ALL) & (diseasename == ALL):
        healthCenter_fliter = hcdf
    elif (icpc2 == ALL):
        healthCenter_fliter = hcdf[hcdf.Disease_Name == diseasename]
    elif (diseasename == ALL):
        healthCenter_fliter = hcdf[hcdf.ICPC2_Classfication == icpc2]
    else:
        healthCenter_fliter = hcdf[(hcdf.ICPC2_Classfication == icpc2) & 
                                  (hcdf.Disease_Name == diseasename)]
    
    with output:
        display(healthCenter_fliter)
        
#Step-4 : Bind to the event handler to the dropdown value         
def healthCenter_icpc2_eventhandler(change):
    healthCenter_flitering(change.new, icpc2Classfication.value)
def healthCenter_diseasename_eventhandler(change):
    healthCenter_flitering(icpc2Classfication.value, change.new)
    
icpc2Classfication.observe(healthCenter_icpc2_eventhandler, names='value')
diseaseName.observe(healthCenter_diseasename_eventhandler, names='value')

# Step-5: Display the result
display(icpc2Classfication)
display(diseaseName)
display(output)

In [ ]:
hcdf.shape

In [ ]:
hcdf.CP.value_counts()

In [ ]:
#count the missing value
hcdf.count()

Building Histogram for categorical values or objects 

In [ ]:
#building Histogram for categorical values or objects 
@interact 
def hist(x=list(hcdf.select_dtypes('object').columns),
                 #theme=list(cf.themes.THEMES.keys()), 
                 theme=list(cf.getThemes()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    hcdf[x].iplot(kind='hist', x=x, mode='markers', xTitle='x',
                   yTitle='count', title='Distribution',
                   theme=theme, colorscale=colorscale)

Interactive visulization: Inspecting objects or catagorical attributes 

In [ ]:
#Implmentaitng cross tab between two variables 
# Viz x=ICPC2 classfication or ICD 11 calssification and y=CP
@interact 
def crosstab(x=list(hcdf.select_dtypes('object').columns), 
             y=list(hcdf.select_dtypes('object').columns)[1:],
             w=5,h=5,
             theme=list(cf.getThemes()), 
             colorscale=list(cf.colors._scales_names.keys())):
    pd.crosstab(hcdf[x],hcdf[y]).plot.barh(figsize=(w,h)).legend(loc='center left', bbox_to_anchor=(0.3, 0.6))
    plt.xlabel('No. of Count by the Path Followed')

In [ ]:
pd.crosstab(hcdf.ICPC2_Classfication,hcdf.CP).plot.bar()#.legend(loc='uppper left', bbox_to_anchor=(0.8, 0.8)
plt.ylabel('No. of Count')
plt.xlabel('Figure 6. Summary of CPs based on card-sheet')
plt.xticks(rotation=45, horizontalalignment='right',fontsize='medium')
plt.legend(title='Clinical Pathways', bbox_to_anchor=(0.5, 0.64),fontsize='small', frameon=False)
# plt.title('Figure 4: This is a somewhat long figure title', fontsize=16)
plt.rcParams['figure.dpi'] = 150 # default for me was 75
#plt.savefig('Figure 6. Summary of CPs based on card-sheet.pdf',dpi=1)


In [ ]:
pd.crosstab(hcdf.CP,hcdf.ICPC2_Classfication).plot.barh(figsize=(5,5)).legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('No. of Count by the Path Followed')
plt.ylabel('Clincal Pathway')

In [ ]:
pd.crosstab(hcdf.CP,hcdf.ICD11_Classfication).plot.barh(figsize=(7,10)).legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('No. of Count by the Path Followed')
plt.ylabel('Clincal Pathway')
plt.legend(title='Reportable Cases in the Health Center', bbox_to_anchor=(1, 1))

Exploring the frequencey or the summary of based on a certain attributes using catplot. This helps to visulize the frequencey of the flow of refferal based on the three selected data attributes.

In [ ]:
# Visulization of referred reason and age category based on the referred orginazation 
# And also different combination of VIZ also explored 
# x=icpc2 classfication,y=month or year or day and z= cp
@interact 
def catplot(x=list(hcdf.select_dtypes('object').columns), 
            y=list(hcdf.select_dtypes('number').columns)[1:],
            z=list(hcdf.select_dtypes('object').columns)[2:],
            theme=list(cf.getThemes()), 
            colorscale=list(cf.colors._scales_names.keys())):
    sns.catplot(x=x, y=y, hue=z,aspect=3,kind="swarm",  data=hcdf);


In [ ]:
sns.catplot(x="CP", y="Month", hue="ICD11_Classfication",kind="box",dodge=True,
            height=8, aspect=1.5,  data=hcdf,legend=False);
plt.xticks(rotation=45, horizontalalignment='right',fontweight='light',fontsize='x-large')
plt.legend(title='Reportable Cases in the Health Center', bbox_to_anchor=(1, 1))
plt.xlabel('Finding or Path Followed')

In [ ]:
sns.catplot(x="CP", y="Month", hue="ICPC2_Classfication",kind="box",dodge=True,
            height=8, aspect=1.5,  data=hcdf,legend=False);
plt.xticks(rotation=45, horizontalalignment='right',fontweight='light',fontsize='x-large')
plt.legend(title='Reportable Cases in the Health Center', bbox_to_anchor=(1, 1))
plt.xlabel('Finding or Path Followed')

# Part -II: CG-Generated Clincal Data

1. Import the CSV generated data

In [ ]:
cgdf = pd.read_csv("CG_Generated-Table 1.csv", encoding='utf-8')

In [ ]:
#counting values for misssing value detection
cgdf.count()

Building Histogram

In [ ]:
#building Histogram for categorical values or objects 
@interact 
def hist(x=list(cgdf.select_dtypes('object').columns),
                 #theme=list(cf.themes.THEMES.keys()), 
                 theme=list(cf.getThemes()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    cgdf[x].iplot(kind='hist', x=x, mode='markers', xTitle='x',
                   yTitle='count', title='Distribution',
                   theme=theme, colorscale=colorscale)

Interactive visulization: Inspecting objects or catagorical attributes 

In [ ]:
#Implmentaitng cross tab between two variables 
# e.g. x is the disease and y is the path followed 
@interact 
def crosstab(x=list(cgdf.select_dtypes('object').columns), 
             y=list(cgdf.select_dtypes('object').columns)[1:],
             theme=list(cf.getThemes()), 
             colorscale=list(cf.colors._scales_names.keys())):
    pd.crosstab(cgdf[x],cgdf[y]).plot.barh().legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel('Count by the Path Followed')
   

### updated

In [ ]:
cgdfupdated = pd.read_csv("CG_Generated_P2_Updated.csv", encoding='utf-8')
cgdfupdated = pd.read_csv("CG_Generated_P2-Updated1.csv", encoding='utf-8')

In [ ]:
cgdf.shape

In [ ]:
#cgdfupdated.info()
cgdfupdated.CP.value_counts()

In [ ]:
#cgdfupdated['CP'].value_counts().head(30).plot(kind='barh', figsize=(20,10))
cgdfupdated.shape

In [ ]:
#Implmentaitng cross tab between two variables 
# e.g. x is the disease and y is the path followed 
@interact 
def crosstab(x=list(cgdfupdated.select_dtypes('object').columns), 
             y=list(cgdfupdated.select_dtypes('object').columns)[1:],
             theme=list(cf.getThemes()), 
             colorscale=list(cf.colors._scales_names.keys())):
    pd.crosstab(cgdfupdated[x],cgdfupdated[y]).plot.barh(figsize=(7,10)).legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel('Count by the Path Followed')
    #plt.tight_layout()
    #plt.figure(figsize=(30,30))

In [ ]:
pd.crosstab(cgdfupdated.ICPC2_Classfication,cgdfupdated.CP).plot.bar()#.legend(loc='uppper left', bbox_to_anchor=(0.8, 0.8)
plt.ylabel('No. of Count')
plt.xlabel('Figure 8. Summary of CPs based on CGs')
plt.xticks(rotation=45, horizontalalignment='right',fontsize='medium')
plt.legend(title='Clinical Pathways', bbox_to_anchor=(0.45, 0.64), fontsize='small', frameon=False)
plt.savefig('Figure 8. Summary of CPs based on CGs.pdf')

## Comparision: A case of Fever and Pregancy 

In [ ]:
#fever extracted from the CGS
feverCGs=pd.read_csv("Fever_CGS-Table 1.csv", encoding='utf-8')

#fever collected from the card-sheet 
feverCardsheet=pd.read_csv("Fever_CardSheet-Table 1.csv", encoding='utf-8')

#pregnancy extracted from the CGS
pregancyCGs=pd.read_csv("Pregnacy_FullCGs-Table 1.csv", encoding='utf-8')

#pregnancy collected from the card-sheet 
pregnancyCardheet=pd.read_csv("Pregnacy_FullCardsheet-Table 1.csv", encoding='utf-8')

In [ ]:
feverCGs.head(1)

In [ ]:
pd.crosstab(feverCGs.DiseaseName,feverCGs.CP).plot.bar()#.legend(loc='uppper left', bbox_to_anchor=(0.8, 0.8)
plt.ylabel('No. of Count')
plt.xlabel('Fever Clinical Pathways Based on CGs')
plt.xticks(rotation=5, horizontalalignment='right',fontsize='medium')
plt.legend(title='Clinical Pathways', bbox_to_anchor=(0.5, 0.64))

In [ ]:
pd.crosstab(feverCardsheet.Disease_Name,feverCardsheet.CP).plot.bar()#.legend(loc='uppper left', bbox_to_anchor=(0.8, 0.8)
plt.ylabel('No. of Count')
plt.xlabel('Fever Clinical Pathways Based on Card-sheet')
plt.xticks(rotation=5, horizontalalignment='right',fontsize='medium')
plt.legend(title='Clinical Pathways', bbox_to_anchor=(0.5, 0.64))

In [ ]:
pd.crosstab(pregancyCGs.ICPC2_Classfication,pregancyCGs.CP).plot.bar()#.legend(loc='uppper left', bbox_to_anchor=(0.8, 0.8)
plt.ylabel('No. of Count')
plt.xlabel('Figure 9. Extracted CPs from the CGs')
plt.xticks(rotation=5, horizontalalignment='right',fontsize='medium')
plt.legend(title='Clinical Pathways', bbox_to_anchor=(0.5, 0.64), fontsize='small', frameon=False)
plt.savefig('Figure 9. Extracted CPs from the CGs.pdf')

In [ ]:
pd.DataFrame({'Percentage from CGs (%)': pregancyCGs.groupby(('CP')).size() / len(pregancyCGs),'Percentage from Card-sheet (%)': pregnancyCardheet.groupby(('CP')).size() / len(pregnancyCardheet)})

In [ ]:
pd.DataFrame({'Percentage from Card-sheet': pregancyCGs.groupby(('CP')).size() / len(pregancyCGs)})

In [ ]:
pregancyCGs['CP'].value_counts()

In [ ]:
pregancyCGs.shape

In [ ]:
pd.DataFrame({'Percentage from Card-sheet': pregnancyCardheet.groupby(('CP')).size() / len(pregnancyCardheet)})

In [ ]:
pd.crosstab(pregnancyCardheet.ICPC2_Classfication,pregnancyCardheet.CP).plot.bar()#.legend(loc='uppper left', bbox_to_anchor=(0.8, 0.8)
plt.ylabel('No. of Count')
plt.xlabel('Figure 7. Summary of CPs based on card-sheet')
plt.xticks(rotation=5, horizontalalignment='right',fontsize='medium')
plt.legend(title='Clinical Pathways', bbox_to_anchor=(0.55, 0.64),fontsize='small', frameon=False)
#plt.savefig('Figure 7. Summary of CP based on card-sheet.pdf')

In [ ]:
pregnancyCardheet.shape

In [ ]:
pregnancyCardheet

In [ ]:
# Import the necessary libraries first
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)

# Summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(X)
# Summarize selected features
print(features[0:5,:])

In [ ]:
pregnancyCardheet["CP"] = pregnancyCardheet["CP"].astype('category')
pregnancyCardheet.dtypes

In [ ]:
pregnancyCardheet["CP_style_cat"] = pregnancyCardheet["CP"].cat.codes
pregnancyCardheet.head(2)

In [ ]:
sns.catplot(x="CP_style_cat", y="ICPC2_Classfication", hue="CP", kind="swarm", data=pregnancyCardheet);

In [ ]:
sns.catplot(x="ICPC2_Classfication", y="CP", hue="CP_style_cat", kind="swarm", data=pregnancyCardheet);

In [ ]:
sns.catplot(y="CP", hue="ICPC2_Classfication", kind="count",
            palette="pastel", edgecolor=".6",
            data=pregnancyCardheet);


In [ ]:
pregancyCGs["CP"] = pregancyCGs["CP"].astype('category')

In [ ]:
pregancyCGs["CPCGS_style_cat"] = pregancyCGs["CP"].cat.codes
pregancyCGs.head(2)

In [ ]:
sns.catplot(x="CPCGS_style_cat", y="ICPC2_Classfication", hue="CP", kind="swarm",aspect=.5, data=pregancyCGs);

In [ ]:
sns.catplot(x="CP_style_cat", y="ICPC2_Classfication", hue="CP", kind="swarm",aspect=.5, data=pregnancyCardheet);

In [ ]:
sns.barplot(x="ICPC2_Classfication", y="CPCGS_style_cat", data=pregancyCGs)

In [ ]:
sns.catplot(y="CP", hue="ICPC2_Classfication", kind="count",
            palette="pastel", edgecolor=".6",
            data=pregancyCGs);

In [ ]:
sns.catplot(y="CP", hue="ICPC2_Classfication", kind="count",
            palette="pastel", edgecolor=".6",
            data=pregnancyCardheet);

In [ ]:
diseaseList = pregnancyCardheet.CP
countDiseaseList = diseaseList.value_counts()
plt.title('Summary', fontsize=14)
plt.xlabel('Pregnancy Clinical Pathways ', fontsize=14)
plt.ylabel('Number of Cases', fontsize=14)
plt.grid(True)
countDiseaseList.plot(kind='bar')
plt.xticks(rotation=45, horizontalalignment='right',fontweight='light')

### count and summarized CP [both the cardsheet and CGs]

In [ ]:
countSummaryFROMCG=cgdfupdated.groupby(["ICPC2_Classfication", "CP"]).size()
countSummaryFROMCardSheet=hcdf.groupby(["ICPC2_Classfication", "CP"]).size()

In [ ]:
countSummaryFROMCG

In [ ]:
countSummaryFROMCardSheet

###  Save to Excel and Import for Visulization

In [ ]:
importCPCountSummary = pd.read_csv("CountComparision-Table 1.csv", encoding='utf-8')

In [ ]:
importCPCountSummary.CP.nunique()

In [ ]:
# Display and Check the reuslt 
#new = old[['A', 'C', 'D']].copy()
copyCGfrom = importCPCountSummary[['ICPC2Classification', 'CP', 'CP_in_Cardsheet']].copy()
copyCSfrom = importCPCountSummary[['ICPC2Classification', 'CP', 'CP_in_CGs']].copy()

In [ ]:
importCPCountSummary.info()

In [ ]:
importCPCountSummary

In [ ]:
importCPCountSummary.plot(x ='ICPC2Classification', y=['CP_in_Cardsheet','CP_in_CGs'], kind = 'bar', figsize=(15,7))

In [ ]:
selectedP= importCPCountSummary.loc[importCPCountSummary['ICPC2Classification'] == 'Pregnancy, Childbearing,Family Planning']
selectedPT = cgdfupdated.loc[cgdfupdated['DiseaseName'] == 'AFI']

In [ ]:
selectedP.plot(x ='CP', y=['CP_in_Cardsheet','CP_in_CGs'], kind = 'barh', figsize=(7,5))

In [ ]:
selectedPT.head(1)

In [ ]:
selectedPT['CP'].nunique()

In [ ]:
diseaseList = selectedPT.CP
countDiseaseList = diseaseList.value_counts()
plt.title('Summary', fontsize=14)
plt.xlabel('Followed Clinical Pathways ', fontsize=14)
plt.ylabel('Number of Cases', fontsize=14)
plt.grid(True)
countDiseaseList.plot(kind='bar')
plt.xticks(rotation=45, horizontalalignment='right',fontweight='light')

In [ ]:

AggregatedCPSummary = pd.read_csv("AggregatedSummary-Table 1.csv", encoding='utf-8')

In [ ]:
AggregatedCPSummary

In [ ]:
AggregatedCPSummary.plot(x ='CP_Summary', y=['CP_Collected_From_Cardsheet','CP_Extracted_From_CGs'], kind = 'bar')
plt.xticks(rotation=45, horizontalalignment='right',fontweight='light')
plt.ylabel('No. of Count')
plt.xlabel('Clincal Pathway Followed')

# Part III - Comparision 

# Next steps

Building an algorithm for generating the number of possible paths or patterns for detecting the patient flow treatment and referral paths or matching. Also, able to detect and explore findings that point to more than one disease called "consideration path or crossroad PLUS(+) treatable or referral"